# Data Storage

#### Text Files

The simplest way to store data is with a text file. Reading and writing to a text file from Python is simple [link], and those files can easily be opened with any text editor. A text file can take any form, but your file needs to be parsable. That means when fetching data from the file you (or more commonly, a computer program) needs to know where one data point ends and the next begins.  The standard convention for storing data is by comma sparated value (or CSVs). CSVs store data the same way an excel spreadsheet does, with rows and columns. Let's say you had three rows and three columns of integer values to store. You would store that data in a CSV as:
````
1,2,2
3,2,1
4,4,3
````
Rows are separated by a new line, and columns by commas. Knowing this simple convention makes it easy for a computer to parse the file so that it can then do something with the data.

However, in some cases CSVs are actually a bad choice for storing data. Suppose that one of the fields contains names of bands:
````
ACDC,2,1
Lady Gaga,4,3
Peter,Paul and Marry,1,3
````
Are "Peter" and "Paul and Marry" two different columns? It's obvious to most people that they're not, but not at all obvious to a computer. If you're storing strings and there's chance that you may have commas in those strings, CSVs aren't going to work. Luckily, there are plenty of other more obscure characters that can be used to separatate values. If commas aren't a good choice, then you may choose to go with tabs (represented by \t):
````
ACDC\t2\t1
Lady Gaga\t4\t3
Peter, Paul and Marry\t1\t3
````

You could easily write your own CSV Python parser, but a number of Python tools already exist to do this for you. Let's look at how you'd do this with pandas.

In [2]:
import pandas as pd
file_path = 'artists.csv'
artists_df = pd.read_csv(file_path)

In [6]:
artists_df.head()

,name,musicbrainz_id,id
0,The Scenic,a8e347d8-29d8-459c-a444-68febeb99c6b,500
1,Our Family Name,fc93c97f-1b9b-49af-8370-7b36e29ad22e,710
2,Khamelien,NaN,744
3,Death in the Park,000c60eb-a2c4-46be-9097-e603fd8795c6,786
4,enter the collector,f50c0834-e2a7-4241-bd32-061d67560c41,818


If you open artists.csv in a text editor, the first few lines look like this:
````
name,musicbrainz_id,id
The Scenic,a8e347d8-29d8-459c-a444-68febeb99c6b,500
Our Family Name,fc93c97f-1b9b-49af-8370-7b36e29ad22e,710
Khamelien,,744
````
There are two things worth noting here:

1. Khamelien doesn't have a musicbrainz id, but we still need to leave a space for it in the csv. Every row must have the same number of columns, and if we leave a field blank, Pandas will fill it in with the special NaN (Not a Number) value to denote that it's missing.
2. By default, pandas assumes that the first line of a CSV contains the column names. If your csv does not have column names in the first line, you can set them yourself with the `names` parameter in the read_csv method:

In [7]:
artists_df = pd.read_csv('artists_no_column_names.csv',header=None,names=['name','musicbrainz_id','NBS_id'])
artists_df.head()

,name,musicbrainz_id,NBS_id
0,The Scenic,a8e347d8-29d8-459c-a444-68febeb99c6b,500
1,Our Family Name,fc93c97f-1b9b-49af-8370-7b36e29ad22e,710
2,Khamelien,NaN,744
3,Death in the Park,000c60eb-a2c4-46be-9097-e603fd8795c6,786
4,enter the collector,f50c0834-e2a7-4241-bd32-061d67560c41,818


As mentioned before, separating strings with commas can be problematic. It is better to use tabs (\t). We can use the same `read_csv` pandas method to open tab separated files, but in this case we need to specify that the separaters are tabs using the `sep` parameter.

In [12]:
artists_df = pd.read_csv('artists.tsv',sep='\t')
artists_df.head()

,name,musicbrainz_id,id
0,The Scenic,a8e347d8-29d8-459c-a444-68febeb99c6b,500
1,Our Family Name,fc93c97f-1b9b-49af-8370-7b36e29ad22e,710
2,Khamelien,NaN,744
3,Death in the Park,000c60eb-a2c4-46be-9097-e603fd8795c6,786
4,enter the collector,f50c0834-e2a7-4241-bd32-061d67560c41,818


Now let's suppose we've made some updates to the data, and now we want to save our results to a new csv.

In [15]:
artists_df.loc[4,'name'] = "Enter the Collector"
artists_df.head()

,name,musicbrainz_id,id
0,The Scenic,a8e347d8-29d8-459c-a444-68febeb99c6b,500
1,Our Family Name,fc93c97f-1b9b-49af-8370-7b36e29ad22e,710
2,Khamelien,NaN,744
3,Death in the Park,000c60eb-a2c4-46be-9097-e603fd8795c6,786
4,Enter the Collector,f50c0834-e2a7-4241-bd32-061d67560c41,818


In [16]:
artists_df.to_csv('artists_updated.tsv',sep='\t')

Using the `to_csv` method, we write our results to the file `artists_updated.tsv` which can be opened with the `read_csv` method (using the sep='t' paramater) or with any text editor. Note: if we write to a file that already exists, `to_csv` will overwrite the old file with the new.

#### SQL

Text files work well for simple, relatively small, datasets that can live in one or two independant tables, but when the data you're working with is either really big or you have multiple tables that are related to eachother, you'll need to start looking into other solutions. 

The most common tabular step up from CSVs are relational databases that use SQL (Structured Query Language). Two big advantages of a SQL database over text files are that:

1. It's much easier to join two or more tables that share common columns (we'll see an example in a second)
2. SQL is very efficient at searching through large data sets and pulling data from multiple tables matching a specific query criteria.

There are several different implementations of SQL, and at most companies there is a person (or team of people) who manage the setup and maintenance of a SQL databse. Fortunately, the query languages across all implementations of SQL are very similar, so you don't necessarily need to know the nuts and bolts of how the data is stored to get the data you need.

If you're new to SQL and don't have the resources to manage a production-grade relational database, there's a free and easy-to-use SQL implementation called sqlite. It requires no setup and comes pre-installed with Python. sqlite is a standalone application that you can access from the command line [link] by typing `sqlite3 path_to_database` (where `path_to_database` is the path to a database file. In your terminal window, change directories [cd] to where the database file `nbs.db` is located and run
````
$ sqlite3 nbs.db
````

You should see a command prompt letting you know that you're using the sqlite command-line program. To see what tables are included in the nbs database use the `.tables` command:
````
sqlite> .tables
artists              networks
endpoints            x_artists_endpoints
````
and to see the columns and types for a table use the `.schema` command:
````
sqlite> .schema artists
CREATE TABLE "artists" (
"index" INTEGER,
  "name" TEXT,
  "musicbrainz_id" TEXT,
  "created_at" TIMESTAMP,
  "updated_at" TIMESTAMP,
  "deleted_at" TIMESTAMP,
  "id" INTEGER
);
CREATE INDEX "ix_artists_index"ON "artists" ("index");
````
Now let's query some data. To get the first five rows from the `artists` table, run the following from the command line:
````
sqlite> SELECT * FROM artists LIMIT 5;
````
The results should look like this:
````
The Scenic|a8e347d8-29d8-459c-a444-68febeb99c6b|2009-06-16 07:58:58|2015-05-19 12:28:17||500
Our Family Name|fc93c97f-1b9b-49af-8370-7b36e29ad22e|2009-06-16 08:04:18|2014-08-27 09:04:37||710
Khamelien||2009-06-16 08:05:08|2014-08-27 09:04:36||744
Death in the Park|000c60eb-a2c4-46be-9097-e603fd8795c6|2009-06-16 08:06:10|2014-08-27 09:04:37||786
enter the collector|f50c0834-e2a7-4241-bd32-061d67560c41|2009-06-16 08:06:54|2014-08-27 09:04:34||818
````
To get the first five rows that don't have a musicbrainz id:
````
sqlite> SELECT * FROM artists WHERE musicbrainz_id IS NULL LIMIT 5;
Khamelien||2009-06-16 08:05:08|2014-08-27 09:04:36||744
Shorelines End||2009-06-16 08:10:11|2014-08-27 09:04:32||952
Dearth||2009-06-16 08:36:24|2014-08-27 09:51:29||1978
Redink||2009-06-16 19:45:19|2014-08-27 09:59:37||4297
The Atomic Ballroom Calamity||2009-06-16 19:47:00|2014-08-27 09:59:45||4359
````
and to find an artist named "Fuck Montreal":
````
sqlite> SELECT * FROM artists WHERE name='Fuck Montreal';
Fuck Montreal|87cf6aa6-a005-445b-8920-1c5b3fdfbfaa|2009-06-17 02:21:37|2014-08-27 09:43:26||28561
````

In addition to the sqlite command-line tool, we can also query a sqlite database directly from Python. As with CSVs, there are a couple of tools for doing this, and we'll use a tool built directly into pandas. Because pandas is tabular, pulling data from sql and putting it directly into a dataframe is fairly simple.

(Note that in the below query you will have to update the url to reflect the path to where nbs.db is stored on your computer.)

In [25]:
import sqlite3 as lite
import pandas as pd
con = lite.connect('/Users/adamhajari/Projects/python/tutorials/databases/nbs.db')
query = "select * from artists"
artists_df = pd.read_sql(query,con)
con.close()
print len(artists_df)
artists_df.head()


4349


,name,musicbrainz_id,created_at,updated_at,deleted_at,id
0,The Scenic,a8e347d8-29d8-459c-a444-68febeb99c6b,2009-06-16 07:58:58,2015-05-19 12:28:17,None,500
1,Our Family Name,fc93c97f-1b9b-49af-8370-7b36e29ad22e,2009-06-16 08:04:18,2014-08-27 09:04:37,None,710
2,Khamelien,None,2009-06-16 08:05:08,2014-08-27 09:04:36,None,744
3,Death in the Park,000c60eb-a2c4-46be-9097-e603fd8795c6,2009-06-16 08:06:10,2014-08-27 09:04:37,None,786
4,enter the collector,f50c0834-e2a7-4241-bd32-061d67560c41,2009-06-16 08:06:54,2014-08-27 09:04:34,None,818


Here we've created a connection to the sqlite database using the sqlite3 library, and then passed a sql query and our sqlite connection object to the read_sql method to get the query results as a dataframe.

**Exercise:** Pandas' `read_sql` method is still in development and does not support all sql database engines. For other engines (like MySQL) you may have to write you own function to turn a sql query into a pandas dataframe. As an exercise, try doing this for sqlite. Your function will take a query string and a sqlite connection object and return a pandas dataframe. You can use any methods from the sqlite module, but use only the pandas dataframe constructor from the pandas library (pd.DataFrame()). 

*Hint: Here's one way to get column names from from a sqlite query: http://stackoverflow.com/questions/7831371/is-there-a-way-to-get-a-list-of-column-names-in-sqlite*

#### Non-tabular Data

So far, all of the ways we've looked at for storing data have assumed that our data is tabular, but sometimes data doesn't fit neatly into rows and columns. Suppose for instance that you had a list of arists and you wanted to store all of their EPs and all of the tracks on those EPs. It is *possible* to store this data is a relational database, but that's not necessarily the best or most intuitive way to do so. 

An alternative data structure uses key-value mappings. Storing EP data for the bands Green Day and Alkaline Trio in a key-value map might look like this:

In [2]:
ep_data = {"Green Day":{
    "EPs":{
        "1,000 Hours":{
             "release_year":1989,
             "tracks":["1,000 Hours","1,000 Hours","Only of You","The One I Want"]},
        "Slappy":{
             "release_year":1990,
             "tracks":["Paper Lanterns","Why do you Want Him?","409 in Your Coffeemaker","Knowledge"]},
        "Tune In, Tokyo...":{
         "release_year":2001,
         "location":"Japan"}
        },
    "artist_id":1},
 "Alkaline Trio":{
    "EPs":{
        "For Your Lungs Only":{
             "release_year":1998,
             "tracks":["Snake Oil Tanker","Southern Rock","Cooking Wine","For Your Lungs Only"]},
        "I Lied My Face Off":{
             "release_year":1999,
             "tracks":["Goodbye Forever","This Is Getting Over You","Bleeder","I Lied My Face Off"]},
        "Broken Wing":{
             "release_year":2013,
             "tracks":["Balanced On A Shelf","Pocket Knife","Broken Wing","Sun Burns"]}
        },
    "artist_id":2}}


The key-value pair data structure in Python is called a dictionary (and known as a JSON object in javascript, and goes by other names in other programming languages). Python dictionaries are very flexible. The key can be either a string or a number, and the value can be almost anything. This flexibility is the advantage of dictionaries over tabular data structures like tables or dataframes. 

If we want all of the metadata available for the Alkaline Trio album "For Your Lungs Only" we just pass in the keys for the nested set of dictionaries:

In [3]:
ep_data['Alkaline Trio']['EPs']['For Your Lungs Only']

{'release_year': 1998,
 'tracks': ['Snake Oil Tanker',
  'Southern Rock',
  'Cooking Wine',
  'For Your Lungs Only']}

and to get just the release year for that album:

In [4]:
ep_data['Alkaline Trio']['EPs']['For Your Lungs Only']['release_year']

1998

To store a Python dictionary, we can convert to a JSON string and safe to a text file:

In [11]:
import json
ep_data_str = json.dumps(ep_data)
f = open('ep_data.json', 'w')
f.write(ep_data_str)
f.close()

After converting the dictionary to a string (using `json.dumps`), the above code block writes that string to the file `ep_data.json` (the last three lines).

#### MongoDB

Storing key-value data as json files is fine for small data sets (less than a few Mb), but once your data grows larger than a 100 Mb, you may want to consider moving it into a queryable storage engine like MongoDB.

MongoDB has a Python library and a good tutorial for getting started [link: http://api.mongodb.org/python/current/tutorial.html]. 